<a href="https://colab.research.google.com/github/garg-aayush/Pytorch_Examples/blob/master/01_CNN_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### In this example, we build a 2-convolution layer network to classify the hand-written digits of the MNIST dataset

In [0]:
## import the required modules 
import torch
import torch.nn as nn
import torchvision 
import torchvision.transforms as transforms

# for plotting
from bokeh.plotting import figure
from bokeh.io import show
from bokeh.models import LinearAxis, Range1d
import numpy as np

from bokeh.io import output_notebook

# Call once to configure Bokeh to display plots inline in the notebook.
output_notebook()

In [0]:
# Check for the device, whether GPU is available or not
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [0]:
# Make sure that google drive is mounted
# Create the folders, if required
DATAPATH = '/content/drive/My Drive/DATASETS/'
MODELPATH = '/content/drive/My Drive/MODELS/'


In [0]:
## Set the input hyperparameters
num_epochs = 6                 # number of times go through the data
num_classes = 10                # number of output classes (MNIST: images of digits from 0 to 9)
batch_size = 50                 # number of input/output pairs to use for updating the weights (mini-batch approach)
learning_rate = 0.001           # alpha :  w_new = w_old - alpha * dw

In [0]:
# Set the transforms to apply to data
trans_list = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

## Check,if necessary 
# trans_list

In [0]:
# Load the MNIST dataset
train_dataset = torchvision.datasets.MNIST(root=DATAPATH, train=True, transform=trans_list, download=True)
test_dataset = torchvision.datasets.MNIST(root=DATAPATH, train=False, transform=trans_list, download=False)

## You can check the downloaded and transformed data
# print(train_dataset)
# print(test_dataset)

In [0]:
## Make the DataLoader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [0]:
## Make the CNN network class
class ConvNet(nn.Module):

  def __init__(self):
    ## Make the different layers for the Network
    super(ConvNet,self).__init__()
    # make 1st convolution layer: 32 channel 5X5 filter --> ReLU activation --> Maxpooling
    self.layer1 = nn.Sequential(nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
                                nn.ReLU(),
                                nn.MaxPool2d(kernel_size=2, stride=2))
      
    # make 2nd convolution layer: 64 channel 5X5 filter --> ReLU activation --> Maxpooling
    self.layer2 = nn.Sequential(nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
                                nn.ReLU(),
                                nn.MaxPool2d(kernel_size=2, stride=2))
      
    # define dropout action
    self.drop_out = nn.Dropout()      # use default dropout prob. of p=0.5

    # make the fully-connected layers
    self.fc1 = nn.Linear(7*7*64, 1000)
    self.fc2 = nn.Linear(1000, 10)

  def forward(self,x):
    ## Use the layers to make the complete network
    out = self.layer1(x)
    out = self.layer2(out)
    
    # flatten thhe output
    out = out.reshape(out.size(0), -1)

    out = self.drop_out(out)
    out = self.fc1(out)
    out = self.fc2(out)

    return out

In [0]:
# Create the network instance
model = ConvNet().to(device)

# set the loss-criterion
criterion = nn.CrossEntropyLoss()     # Combines the softmax fn and the -inve log-likelihood fn

# set the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [0]:
# Check the model, criterion and optimizer
print(model)
print(criterion)
print(optimizer)

ConvNet(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (drop_out): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=3136, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=10, bias=True)
)
CrossEntropyLoss()
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)


In [0]:
## Train the model

total_step = len(train_loader)      # number of steps in one epoch 
loss_list = []                      # list to save the losses
acc_list = []                       # list to save and check the accuracy at each step


for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    
    # transfer to GPU if device CUDA
    images = images.to(device)
    labels = labels.to(device)

    # Forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss_list.append(loss.item())         # .item() loss is a scaler tensor quantity

    # Backward pass to optimize and update the weights
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Check the accuracy
    total = labels.size(0)                        # as shape (num. of examples, ....)
    _, predicted = torch.max(outputs.data,1)      # as .max returns value, index. Here, index corresponds number where value is 1
    correct = (predicted == labels).sum().item()
    acc_list.append(correct / total)

    # print the information at every 100th step
    if (i+1) % 100 == 0:
      print('Epoch [{}/{}], Step[{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
            .format(epoch +1, num_epochs, i+1, total_step, loss.item(), (correct / total)*100))



Epoch [1/6], Step[100/1200], Loss: 0.2614, Accuracy: 94.00%
Epoch [1/6], Step[200/1200], Loss: 0.2287, Accuracy: 92.00%
Epoch [1/6], Step[300/1200], Loss: 0.0875, Accuracy: 98.00%
Epoch [1/6], Step[400/1200], Loss: 0.0675, Accuracy: 96.00%
Epoch [1/6], Step[500/1200], Loss: 0.1383, Accuracy: 94.00%
Epoch [1/6], Step[600/1200], Loss: 0.2320, Accuracy: 94.00%
Epoch [1/6], Step[700/1200], Loss: 0.0790, Accuracy: 98.00%
Epoch [1/6], Step[800/1200], Loss: 0.0319, Accuracy: 98.00%
Epoch [1/6], Step[900/1200], Loss: 0.1234, Accuracy: 96.00%
Epoch [1/6], Step[1000/1200], Loss: 0.2010, Accuracy: 98.00%
Epoch [1/6], Step[1100/1200], Loss: 0.0339, Accuracy: 100.00%
Epoch [1/6], Step[1200/1200], Loss: 0.1690, Accuracy: 94.00%
Epoch [2/6], Step[100/1200], Loss: 0.0475, Accuracy: 96.00%
Epoch [2/6], Step[200/1200], Loss: 0.2989, Accuracy: 90.00%
Epoch [2/6], Step[300/1200], Loss: 0.0804, Accuracy: 98.00%
Epoch [2/6], Step[400/1200], Loss: 0.1537, Accuracy: 96.00%
Epoch [2/6], Step[500/1200], Loss: 0

In [0]:
# Test the model
model.eval()      # Freezes batch normalisation, dropout etc 
with torch.no_grad():
  correct = 0
  total = 0
  for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  print('Test Accuracy of the model on the 10000 test images: {} %'.format((correct / total) * 100))

Test Accuracy of the model on the 10000 test images: 98.97 %


In [0]:
# Plot the accuracy and loss
p = figure(y_axis_label='Loss', width=850, y_range=(0, 1), title='PyTorch ConvNet results')
p.extra_y_ranges = {'Accuracy': Range1d(start=0, end=100)}
p.add_layout(LinearAxis(y_range_name='Accuracy', axis_label='Accuracy (%)'), 'right')
p.line(np.arange(len(loss_list[0:-1:10])), loss_list[0:-1:10])
p.line(np.arange(len(loss_list[0:-1:10])), np.array(acc_list[0:-1:10]) * 100, y_range_name='Accuracy', color='red')
show(p)

In [0]:
# Save the model checkpoint
torch.save(model.state_dict(), MODELPATH + 'model_01_cnn.ckpt')

# New Section